In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md
import pandas as pd
from numba import jit
from math import floor

In [ ]:
@jit(nopython=True) #numba to speed up our brute force approach
def periodic_neighbours(pos, maxdist, L,i):
    maxdistsq = maxdist**2
    #rL = 1./L
    bonds = []
    dists = []
    tar_pos = pos[i]
    for j in range(len(pos)):
        if j != i:
            #for j in range(i+1, len(pos)):
            distsq = 0
            for d in range(pos.shape[1]):
                #print(d)
                diff = tar_pos[d] - pos[j,d]
                diff -= L[d] * floor(diff * (1.0/L[d]) + 0.5)
                distsq += diff*diff
            if distsq <= maxdistsq and distsq > 0.18**2:
                #print(distsq,maxdistsq,i,j)
                #bonds.append(i)
                bonds.append(j)
                dists.append(distsq)
    return np.array(bonds, np.int64)

In [ ]:
temp = md.load("largeice.gro")


In [ ]:
pos = temp.xyz[0]#[traj.top.select("name O")]
dt = temp.topology.to_dataframe()[0]
bv =temp.unitcell_lengths[0]
#allindx = periodic_neighbours(pos,0.6,bv,-1) + 1
#oindx = [i for i in allindx if dt[dt.serial == i ].element.item() == "O"]
#seq = [dt[dt.serial == i].resSeq.item() for i in oindx]

In [ ]:
@jit(nopython = True)
def periodicNorm(dpos,box_vectors):
    #global box_vectors;
    my_box = box_vectors
    for comp in range(len(dpos)):
        if np.abs(dpos[comp]) > my_box[comp]/2.0:
            dpos[comp] -= np.sign(dpos[comp])*my_box[comp]
            
    return np.sqrt(np.sum(dpos**2))

In [ ]:
def F4(seq,dt,pos,bv,temp):
 
    Oa = temp.top.select(f"resSeq {seq} and name O").item()
    allindx = periodic_neighbours(pos,0.35,bv,Oa) + 1
    oindx = [i for i in allindx if dt[dt.serial == i].element.item() == "O"]
    f4 = 0.0
    for j in oindx:
        Ob = int(j)
        distlist = []

        resseq = dt[dt.serial == Ob].resSeq.item()

        OaHb = temp.top.select(f"resSeq {resseq} and element H")
        for index in OaHb:
            dpos = pos[Oa] - pos[int(index)]
            distlist.append(periodicNorm(dpos, bv))
        resseq = dt[dt.serial == Oa+1].resSeq.item()

        ObHa = temp.top.select(f"resSeq {resseq} and element H")
        for index in ObHa:
            dpos = pos[Ob] - pos[int(index)]
            distlist.append(periodicNorm(dpos, bv))

        an_array = np.zeros(4,dtype = int)
        if min(distlist) == distlist[3] :
            an_array[0] = ObHa[0]
            an_array[1] = Oa
            an_array[2] = Ob - 1 
            if max(distlist) == distlist[0] :
                an_array[3] = OaHb[0]
            else :
                an_array[3] = OaHb[1]
        elif min(distlist) == distlist[2] :
            an_array[0] = ObHa[1]
            an_array[1] = Oa
            an_array[2] = Ob - 1 
            if max(distlist) == distlist[0] :
                an_array[3] = OaHb[0]
            else :
                an_array[3] = OaHb[1]
        elif min(distlist) == distlist[0] :
            an_array[0] = OaHb[1]
            an_array[1] = Ob - 1 
            an_array[2] = Oa
            if max(distlist) == distlist[3] :
                an_array[3] = ObHa[0]
            else :
                an_array[3] = ObHa[1]
        elif min(distlist) == distlist[1] :
            an_array[0] = OaHb[0]
            an_array[1] = Ob - 1 
            an_array[2] = Oa
            if max(distlist) == distlist[3] :
                an_array[3] = ObHa[0]
            else :
                an_array[3] = ObHa[1]
        else:
            print("Error!")
        A , B, C, D = pos[an_array]
        AB = B - A
        BC = C - B
        CD = D - C
        ABC = np.cross(AB, BC)
        BCD = np.cross(BC, CD)
        phi = np.arccos(np.dot(ABC, BCD)/(np.linalg.norm(ABC) * np.linalg.norm(BCD)))
        f4 += np.cos(3*phi)
        
    return (f4 / len(oindx)) 

In [ ]:
F4_list = []
for i in range(1000):
    F4_list.append(F4(i+1,dt,pos,bv,temp))
    print(i)

In [ ]:
def avgF4(seq,dt,pos,bv,temp):
    F4_list = []
    for i in seq:
        F4_list.append(F4(i,dt,pos,bv,temp))
    return pd.Series(np.array(F4_list,dtype = float)).dropna().mean()

In [ ]:
fo = open("avgf4_sigma1.xvg","w")
avgf4 = []
for frame in range(10):
    pos = temp.xyz[frame*10]#[traj.top.select("name O")]
    dt = temp.topology.to_dataframe()[0]
    bv =temp.unitcell_lengths[frame*10]
    allindx = periodic_neighbours(pos,0.6,bv,-1) + 1
    oindx = [i for i in allindx if dt[dt.serial == i ].element.item() == "O"]
    seq = [dt[dt.serial == i].resSeq.item() for i in oindx]
    avgf = avgF4(seq,dt,pos,bv,temp)
    avgf4.append(avgf)
    fo.write(f"{avgf}\n")
    print(frame)
fo.close()

In [ ]:
plt.hist(F4_list)

In [ ]:
import plotly.graph_objects as go
from numpy import *
from scipy.linalg import norm

In [ ]:
def get_coords(r,x,y,z):
    theta = linspace(0,2*pi,100)
    phi = linspace(0,pi,100)
    x = x + r* outer(cos(theta),sin(phi))
    y = y + r* outer(sin(theta),sin(phi))
    z = z + r* outer(ones(100),cos(phi))  # note this is 2d now
    return x , y, z

In [ ]:
def get_Coords_cyllinder(p0, p1, R = 1):
    
    #vector in direction of axis
    v = p1 - p0
    #find magnitude of vector
    mag = norm(v)
    #unit vector in direction of axis
    v = v / mag
    #make some vector not in the same direction as v
    not_v = np.array([1, 0, 0])
    if (v == not_v).all():
        not_v = np.array([0, 1, 0])
    #make vector perpendicular to v
    n1 = np.cross(v, not_v)
    #normalize n1
    n1 /= norm(n1)
    #make unit vector perpendicular to v and n1
    n2 = np.cross(v, n1)
    #surface ranges over t from 0 to length of axis and 0 to 2*pi
    t = np.linspace(0, mag, 10)
    theta = np.linspace(0, 2 * np.pi, 10)
    #use meshgrid to make 2d arrays
    t, theta = np.meshgrid(t, theta)
    #generate coordinates for surface
    X, Y, Z = [p0[i] + v[i] * t + R * np.sin(theta) * n1[i] + R * np.cos(theta) * n2[i] for i in [0, 1, 2]]
    return X, Y, Z


In [ ]:
OaHb

In [ ]:
fig = go.Figure()                            

colorscale = [[0, 'blue'],
             [1, 'blue']]
for i in an_array:
    x , y, z = get_coords(0.025,*pos[int(i)])

    fig.add_trace(go.Surface(
        x=x,
        y=y,
        z=z,
        surfacecolor = z*0,
        showlegend = False,
        hoverinfo = "skip",
        showscale = False))
x , y, z = get_coords(0.025,*pos[int(616)])

fig.add_trace(go.Surface(
    x=x,
    y=y,
    z=z,
    surfacecolor = z*1,
    showlegend = False,
    hoverinfo = "skip",
    showscale = False))
x , y, z = get_coords(0.025,*pos[int(4)])

fig.add_trace(go.Surface(
    x=x,
    y=y,
    z=z,
    surfacecolor = z*1,
    showlegend = False,
    hoverinfo = "skip",
    showscale = False))


X , Y , Z  = get_Coords_cyllinder(pos[3], pos[615], R = 0.01)
fig.add_trace(go.Surface(x=X, y= Y, z=Z,
         colorscale = colorscale,showlegend = False,
         showscale=False,hoverinfo = "skip",
         opacity=1))
  
X , Y , Z  = get_Coords_cyllinder(pos[3], pos[5], R = 0.01)
fig.add_trace(go.Surface(x=X, y= Y, z=Z,
         colorscale = colorscale,showlegend = False,
         showscale=False,hoverinfo = "skip",
         opacity=1))
X , Y , Z  = get_Coords_cyllinder(pos[615], pos[617], R = 0.01)
fig.add_trace(go.Surface(x=X, y= Y, z=Z,
         colorscale = colorscale,showlegend = False,
         showscale=False,hoverinfo = "skip",
         opacity=1))  

fig.layout.scene.camera.projection.type = 'orthographic'
fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False, bgcolor='rgba(0,0,0,1)' )
fig.update_layout(showlegend=False)
fig.update_layout(height = 500,width = 500, margin=dict(l=0, r=0, b=0, t=0))
name = 'eye = (x:0.1, y:0.1, z:1.5)'
camera = dict(
    eye=dict(x=0.0, y=0.0, z=2)
)

fig.update_layout(scene_camera=camera, paper_bgcolor = "black",plot_bgcolor = "black")
#fig.write_html("file_run1.html")
fig.show()

